In [0]:
!pip install BeautifulSoup4

In [0]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime

In [0]:
TABLE_NAME = "finance_catalog.db_landing.src_raw_holidays"

In [0]:

url = "https://www.finra.org/filing-reporting/market-transparency-reporting/holiday-calendar"

headers = {"User-Agent": "Mozilla/5.0"}

response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, 'html.parser')

table = soup.find("table")

rows = []
for tr in table.find_all("tr"):
    cells = [td.get_text(strip=True) for td in tr.find_all(["td", "th"])]
    if cells:
        rows.append(cells)

df = pd.DataFrame(rows, columns=["date", "holiday"])
df["date"] = pd.to_datetime(df["date"]).dt.strftime('%Y-%m-%d')

(
    spark
        .createDataFrame(df)
        .write
        .format("delta")
        .mode("append")
        .saveAsTable(TABLE_NAME)
)

In [0]:
# spark.table(TABLE_NAME).write.mode("overwrite").parquet(STORAGE_URL) 

In [0]:
# spark.table(TABLE_NAME).display()
# spark.sql(f"DROP TABLE IF EXISTS {TABLE_NAME}")